In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("cust_churn_preprocessed_data.csv")

In [19]:
# splitting the features and target
x = df[['tenure', 'MonthlyCharges','TotalCharges','TechSupport_Yes','Contract_Two year']]
y = df["Churn_Yes"]

In [20]:
sd=StandardScaler()
X_t=sd.fit_transform(x)

param_grid = {'penalty': ['l1', 'l2', 'elasticnet', 'none'],'C': [0.01, 0.1, 1, 10, 100],'solver': ['liblinear', 'saga', 'lbfgs'],'max_iter': [100, 200, 500]}
grid=GridSearchCV(LogisticRegression(), param_grid, refit=True, verbose=3,n_jobs=-1)
grid.fit(X_t,y)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'max_iter': [100, 200, 500],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['liblinear', 'saga', 'lbfgs']},
             verbose=3)

In [21]:
result=grid.cv_results_
y_test_pred = grid.predict(X_t)
print("Accuracy Score:\n", accuracy_score(y, y_test_pred))
print("Confusion Matrix:\n", confusion_matrix(y, y_test_pred))
print("Classification Report:\n", classification_report(y, y_test_pred))
table=pd.DataFrame.from_dict(result)
table

Accuracy Score:
 0.8124787487249235
Confusion Matrix:
 [[4261  229]
 [ 874  518]]
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.95      0.89      4490
           1       0.69      0.37      0.48      1392

    accuracy                           0.81      5882
   macro avg       0.76      0.66      0.68      5882
weighted avg       0.80      0.81      0.79      5882



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_max_iter,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.020415,0.005357,0.004784,0.004298,0.01,100,l1,liblinear,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l1', ...",0.825828,0.806287,0.806973,0.817177,0.810374,0.813328,0.007345,1
1,0.038717,0.006313,0.002624,0.000811,0.01,100,l1,saga,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l1', ...",0.818182,0.808836,0.804422,0.807823,0.807823,0.809417,0.004630,13
2,0.002820,0.000896,0.000000,0.000000,0.01,100,l1,lbfgs,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l1', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,76
3,0.011310,0.001417,0.002737,0.000827,0.01,100,l2,liblinear,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l2', ...",0.822430,0.806287,0.806122,0.813776,0.801871,0.810097,0.007261,10
4,0.020460,0.002479,0.001674,0.000425,0.01,100,l2,saga,"{'C': 0.01, 'max_iter': 100, 'penalty': 'l2', ...",0.826678,0.806287,0.803571,0.818027,0.809524,0.812818,0.008464,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.002152,0.001432,0.000000,0.000000,100.00,500,elasticnet,saga,"{'C': 100, 'max_iter': 500, 'penalty': 'elasti...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,76
176,0.000808,0.000990,0.000000,0.000000,100.00,500,elasticnet,lbfgs,"{'C': 100, 'max_iter': 500, 'penalty': 'elasti...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,76
177,0.001718,0.003435,0.000000,0.000000,100.00,500,none,liblinear,"{'C': 100, 'max_iter': 500, 'penalty': 'none',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,76
178,0.002686,0.003154,0.000000,0.000000,100.00,500,none,saga,"{'C': 100, 'max_iter': 500, 'penalty': 'none',...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,76


In [22]:
filename="final_model_sales.sav"
pickle.dump(grid,open(filename,'wb'))

In [28]:
input_data = {
    'tenure':2, 
    'MonthlyCharges':29.5,
    'TotalCharges':28.5, 
    'TechSupport_Yes':1, 
    'Contract_Two year':0
}

In [29]:
input_data_df = pd.DataFrame([input_data])
preinput=sd.transform(input_data_df)
loaded_model=pickle.load(open("final_model_sales.sav",'rb'))
prediction=loaded_model.predict(preinput)
pred_prob = loaded_model.predict_proba(preinput)
print(prediction)
print(f"Prediction: {'Churn' if prediction[0] == 1 else 'No Churn'}")
print(f"Prediciton Probability: {pred_prob}")



[0]
Prediction: No Churn
Prediciton Probability: [[0.76098533 0.23901467]]
